evolution-strategies

Copyright (c) 2019 Patrick Deubel

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in
all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
THE SOFTWARE.

evolution-strategies includes:

evolution-strategies-starter
Copyright (c) 2016 OpenAI (http://openai.com)

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in
all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
THE SOFTWARE.

# Evaluation and Visualization of evolution-strategies

This Notebook can be used to easily evaluate and visualize results created with the `evolution-strategies.ipynb` Notebook.

When the functions are used, the explanations are only the basics. Feel free to look into the files in the `utils/` folder where the methods are implemented and documented.

## TrainingRun and Experiment

There are two objects that are mainly used to evaluate the results. An object of type `TrainingRun` describes one execution of the `run_master()` function of the main Notebook. It is therefore one result from the algorithm. `Experiment` objects therefore combine multiple `TrainingRun` objects which have the same values for the configuration. Meaning they have identical `config`, `model_structure` and `optimizations` objects.

In [ ]:
import os

from utils.es_utils import index_training_folder, index_experiments
from utils.config_values import LogColumnHeaders, EvaluationColumnHeaders

from IPython.display import display
from ipywidgets import Video

## Evaluation and Visualization of a TrainingRun

### Creating a TrainingRun object

For the creation of a TrainingRun object one has to pass the folder which was created during the TrainingRun to the `index_training_folder()` function. This will validate all objects in the folder and checks if indeed a valid TrainingRun has been found. If so it is returned.

In [ ]:
training_run = index_training_folder("training_runs/path-to-folder")

### Visualizing the TrainingRun

If you want to visualize the TrainingRun, you can specify the generation which shall be visualized. Note that for that to work, the corresponding model file must be present, otherwise the trained model can not be loaded and therefore no episode can be started. If you omit the generation, or use `-1` the last generation with a model file is used.

Visualizing PyBullet environments is slightly complicated. It takes a lot of time for some reasons and sometimes it does not work. Try changing the timestep limit in `es_utils.rollout_helper()` to decrease the time or try it several times by restarting the Kernel.

In [ ]:
video_file = training_run.visualize(env_seed=None, generation=-1, force=False)

In [ ]:
video = Video.from_file(video_file)
display(video)

### Evaluating the TrainingRun

Since only random evaluations take place during the actual training, the model files should be saved and the evaluation is then done afterwards. This way there are the same amount of evaluations in every generation.

In [ ]:
evaluation = training_run.evaluate(env_seed=None, num_evaluations=5, num_workers=os.cpu_count(), force=False, save=True)

In [ ]:
evaluation

### Plotting a TrainingRun

If you want to plot the reward against the timesteps for example the `plot_training_run` method is useful. It will take the Enums from `LogColumnHeaders` and `EvaluationColumnHeaders` as input to provide a way to uniformely access the data.

The example shows a common plot, the learning curve. Here the x-axis shows the timesteps and the y-axis the reward for a given number of timesteps. For the shaded area the standard deviation between the different evaluations is used. This is only useful when no environment seed was set since if it was set every episode gives the exact same reward.

In [ ]:
fig, ax = training_run.plot_training_run(
    x_value=LogColumnHeaders.TIMESTEPS_SO_FAR,
    y_value=EvaluationColumnHeaders.EVAL_REW_MEAN,
    y_std=EvaluationColumnHeaders.EVAL_REW_STD,
    x_label="Timesteps",
    y_label="Reward",
    plot_label="Experiment Name")

#### Possible values to plot

For a list of possible values to plot you can look into `utils/config_values.py`.

## Evaluation and Visualization of an Experiment

As previously stated, an Experiment is a collection of TrainingRuns with the same values for the configuration. The methods operating on these Experiments are essentially the same as on the TrainingRun but they differ slightly in their meaning. In the following these methods will be presented and the differences get highlighted.

### Creating an Experiment object

To create an Experiment object provide a folder in which the subfolders are TrainingRuns to the `index_experiments()` method. Then they get indexed and a list of Experiments is returned.

In [ ]:
experiments = index_experiments("training_runs/path-to-folder")

In [ ]:
experiments

In [ ]:
experiment = experiments[-1]

### Visualize an Experiment

In the previous cell one experiment was chosen, which gets now used in the further process to demonstrate the methods.

Visualizing an experiment will visualize each TrainingRun the Experiment contains and thus return a list of video_files.

In [ ]:
video_files = experiment.visualize(env_seed=None, generation=-1, force=True)

In [ ]:
for video_file in video_files:
    video = Video.from_file(video_file)
    display(video)

### Evaluate an Experiment

In an evaluation of an Experiment, all TrainingRuns get evaluated. This means that the method `evaluate()` of each TrainingRun will be called. Note that if some TrainingRun objects already got evaluated and the `force` parameter is not set to `True` these individual TrainingRuns do not get reevaluated. This can be a problem if the number of repetitions per generation differs. For a scientific result this number should be the same across the whole evaluation. To be safe just pass `force=True` but remember that this will reevaluate all TrainingRuns and thus can cost some time.

In [ ]:
evaluations = experiment.evaluate(env_seed=None, num_evaluations=5, num_workers=os.cpu_count(), force=True, save=True)

### Plotting an Experiment

Plotting an Experiment is a little different to plotting a TrainingRun. Here the data between the indexed TrainingRun's is gathered and the mean value per Generation is calculated, except for the `y_std` parameter where the standard deviation is calculated. This means that if you want to plot the learning curve and have the standard deviation between the training runs as a shaded area around the curve, you need to provide the mean reward twice: Once for the `y_value` and once for the `y_std` parameter.

In [ ]:
fig, ax = experiment.plot_experiment(
    x_value=LogColumnHeaders.TIMESTEPS_SO_FAR,
    y_value=EvaluationColumnHeaders.EVAL_REW_MEAN,
    y_std=EvaluationColumnHeaders.EVAL_REW_MEAN,
    x_label="Timesteps",
    y_label="Reward",
    plot_label="Experiment Name")